As of Unicode 16.0 (released in September 2024), the standard defines 154,998 characters across 168 scripts
```python
>>> test_string = "hello! こんにちは!"
>>> utf8_encoded = test_string.encode("utf-8")
>>> print(utf8_encoded)
b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
>>> print(type(utf8_encoded))
<class 'bytes'>
>>> # Get the byte values for the encoded string (integers from 0 to 255).
>>> list(utf8_encoded)
[104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129,
161, 227, 129, 175, 33]
>>> # One byte does not necessarily correspond to one Unicode character!
>>> print(len(test_string))
13
>>> print(len(utf8_encoded))
23
>>> print(utf8_encoded.decode("utf-8"))
hello! こんにちは!
```

In [5]:
print("this is a test" + chr(0) + "string")
print(ord("牛"))

this is a test string
29275


When using byte-level tokenization, we do not need to worry about out-of-vocabulary tokens, since we know that any input text can be expressed as a sequence of integers from 0 to
255 by converting our Unicode codepoints into a sequence of bytes (e.g., via the UTF-8 encoding)

In [8]:
with open("data/TinyStoriesV2-GPT4-valid.txt", "r") as f:
    content = f.readlines()

display(content[:10])

for line in content[:10]:
    print(line)

['u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n',
 '<|endoftext|>\n',
 'Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\n',
 'Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\n',
 'Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\n',
 'They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called

u don't have to be scared of the loud dog, I'll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.

<|endoftext|>

Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.

Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."

Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."

They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could h

In [2]:
chr(104)

'h'

In [6]:
type("hello! こんにちは!".encode("utf-8"))

bytes

In [ ]:
ord(b"\xe3")

227

In [28]:
import regex as re

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
re.findall(PAT, "some text that i'll pre-tokenize\n\nand this is a test 1234")

['some',
 ' text',
 ' that',
 ' i',
 "'ll",
 ' pre',
 '-',
 'tokenize',
 '\n',
 '\n',
 'and',
 ' this',
 ' is',
 ' a',
 ' test',
 ' 1234']

In [ ]:
i = 1
if i > 0 and (prev_token := "", next_token := ""):
    print("This is a test")
    print(f"prev_token: {prev_token}, next_token: {next_token}")

This is a test
prev_token: , next_token: 


In [ ]:
import regex as re
from collections import Counter

type TokenCounter = dict[tuple[bytes], int]


def merge_tokens(cur_tokens: TokenCounter, pair_counts: TokenCounter) -> tuple[TokenCounter, bytes]:
    # Find the most frequent pair
    best_pair = max(pair_counts, key=lambda pair: (pair_counts[pair], pair))
    new_token = best_pair[0] + best_pair[1]

    def dec_(pair: tuple[bytes], count: int):
        pair_counts[pair] -= count
        if pair_counts[pair] == 0:
            del pair_counts[pair]

    new_tokens: TokenCounter = Counter()
    for tokens, count in cur_tokens.items():
        # Replace occurrences of best_pair with new_token
        new_sequence: list[bytes] = []
        i = 0
        while i < len(tokens):
            if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == best_pair:
                new_sequence.append(new_token)

                # Update pair counts for adjacent tokens

                if i > 0 and (prev_token := tokens[i - 1],):
                    pair_counts[(prev_token, new_token)] += count
                    dec_((prev_token, best_pair[0]), count)

                if i + 2 < len(tokens) and (next_token := tokens[i + 2],):
                    pair_counts[(new_token, next_token)] += count
                    dec_((best_pair[1], next_token), count)

                i += 2
            else:
                new_sequence.append(tokens[i])
                i += 1
        new_tokens[tuple(new_sequence)] += count

    del pair_counts[best_pair]
    return new_tokens, new_token


def train_bpe_tokenizer(text: str, num_merges: int = 1000, debug: bool = False):
    vocab = ["<|endoftext|>".encode("utf-8")] + [k.to_bytes() for k in range(256)]

    # pre-tokenization
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    pre_tokens: TokenCounter = Counter()
    for match in re.finditer(PAT, text):
        pre_tokens[tuple(b.to_bytes() for b in match.group(0).encode("utf-8"))] += 1

    pair_counts: TokenCounter = Counter()
    for tokens, count in pre_tokens.items():
        for i in range(len(tokens) - 1):
            pair_counts[(tokens[i], tokens[i + 1])] += count

    cur_tokens = pre_tokens
    for iter in range(num_merges):
        if not cur_tokens:
            print("[WARN] No more pairs to merge.")
            break
        if debug:
            print("---")
            print(f"Current pairs:")
            display(pair_counts)

        cur_tokens, new_token = merge_tokens(cur_tokens, pair_counts)
        vocab.append(new_token)

        if debug:
            print(f"Merge {iter + 1}/{num_merges}: {new_token.decode('utf-8')}")
    return vocab


test_texts = """
low low low low low
lower lower widest widest widest
newest newest newest newest newest newest
""".strip()

vocab = train_bpe_tokenizer(test_texts, num_merges=6, debug=True)
print("Vocabulary size:", len(vocab))
print("Sample vocabulary:", vocab[257:])

In [57]:
import os
from typing import BinaryIO


def find_chunk_boundaries(
    file: BinaryIO, desired_num_chunks: int, split_special_token: bytes
) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), "Must represent special token as a bytestring"

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks
    return sorted(set(chunk_boundaries))


## Usage
with open("data/TinyStoriesV2-GPT4-train.10k.txt", "rb") as f:
    num_processes = int(os.cpu_count() * 0.8)
    print(f"Using {num_processes} processes for parallelization.")
    boundaries = find_chunk_boundaries(f, num_processes, "<|endoftext|>".encode("utf-8"))

    # # The following is a serial implementation, but you can parallelize this
    # # by sending each start/end pair to a set of processes.
    # for start, end in zip(boundaries[:-1], boundaries[1:]):
    #     f.seek(start)
    #     chunk = f.read(end - start).decode("utf-8", errors="ignore")
    #     # Run pre-tokenization on your chunk and store the counts for each pre-token

    print("Chunk boundaries:", boundaries)

Using 51 processes for parallelization.
Chunk boundaries: [0, 27984, 56380, 84483, 112534, 140265, 167934, 196587, 223777, 251706, 279680, 309116, 336001, 363424, 391155, 419861, 447438, 475572, 502961, 531361, 558892, 586921, 614908, 643101, 670487, 699071, 726905, 754966, 782830, 810321, 838413, 866522, 894206, 922230, 950039, 978051, 1005989, 1033949, 1061798, 1090104, 1117981, 1145806, 1173275, 1201511, 1229230, 1257207, 1285071, 1313298, 1341654, 1369733, 1398182, 1424708]


In [ ]:
def split_by_special_token(
    chunk: str, special_tokens: list[str] = ["<|endoftext|>"]  # Default special token
) -> list[str]:
    pattern = "|".join(re.escape(token) for token in special_tokens)
    return re.split(pattern, chunk)

In [55]:
!head -n 10000 data/TinyStoriesV2-GPT4-train.txt > data/TinyStoriesV2-GPT4-train.10k.txt

In [56]:
!less data/TinyStoriesV2-GPT4-train.10k.txt

7=
Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  
He said, “Wow, that is a really amazing vase! Can I buy it?” 
The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”
So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was. 
And that's how Ben found an amazing vase in the store!
<|endoftext|>
Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.
One day, Ollie's mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to catch fish. He saw his

In [19]:
from collections import Counter

c = Counter()
c[tuple(b.to_bytes() for b in "hello".encode("utf-8"))] += 1
c

pair = max(c.items(), key=lambda x: (x[1], x[0]))[0]
new_token = b"".join(pair)
new_token

b'hello'

In [ ]:
tuple(b"hello")

(104, 101, 108, 108, 111)

In [23]:
bytes((104))

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [24]:
tuple([0])

(0,)